In [ ]:
pip install instaloader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.8 MB/s eta 0:00:00


In [ ]:
pip install pandas

In [ ]:
import instaloader
import pandas as pd
import time
import random
import os
import re  # Import the regular expression module

# Initialize Instaloader
loader = instaloader.Instaloader()

# Load or login session
username = ''
password = ''
try:
    loader.load_session_from_file(username)
except FileNotFoundError:
    loader.login(username, password)
    loader.save_session_to_file()

# Profile name
profile_name = 'cmrit.bangalore'

# Fetch profile data
try:
    profile = instaloader.Profile.from_username(loader.context, profile_name)
except instaloader.exceptions.ProfileNotExistsException:
    print(f"Profile '{profile_name}' not found.")
    exit()

# Data collection
data = []
try:
    skip_count = 0  # Number of posts to skip
    post_counter = 50  # Number of posts to collect after skipping
    max_posts = 50     # Number of posts to collect in total

    for post_index, post in enumerate(profile.get_posts()):
        if post_index < skip_count:
            continue  # Skip the first `skip_count` posts

        if post_counter >= max_posts:  # Stop after collecting the desired number of posts
            break

        # Check if the post has a caption
        caption = post.caption if post.caption else ""

        # Extract hashtags from the caption
        hashtags = re.findall(r'#\w+', caption)  # Regular expression to find hashtags

        # Collect post data
        post_info = {
            "date": post.date,
            "caption": caption,
            "hashtags": ', '.join(hashtags),  # Store hashtags as a comma-separated string
            "likes": post.likes,
            "comments": post.comments,
            "post_url": post.url
        }
        data.append(post_info)
        post_counter += 1
        time.sleep(random.uniform(61, 63))  # Random delay
except instaloader.exceptions.TooManyRequestsException:
    print("Rate limit reached. Try again later.")

# Save data to CSV
if data:
    output_file = os.path.abspath(f"{profile_name}_instagram_next_200_posts.csv")
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)  # Saving to CSV
    print(f"Data saved to {output_file}")
else:
    print("No data was collected.")

Data saved to cmrit.bangalore_instagram_next_200_posts.csv
